# setup

In [1]:
import torch
import yaml
import random
import numpy as np
import sys

from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader

sys.path.append('../utils')
from dataset import ECG_TEXT_Dsataset
from builder import ECGCLIP
from utils import find_best_thresholds, metrics_table

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
config = yaml.load(open("config.yaml", "r"), Loader=yaml.FullLoader)
model_checkpoints_folder = '../checkpoints/'

In [4]:
torch.manual_seed(42)
random.seed(0)
np.random.seed(0)

# utils

# dataset

In [5]:
# data_path = config['dataset']['data_path']
data_path = '\\Users\katri\Downloads\git\lesaude\code\CODEmel'
dataset = ECG_TEXT_Dsataset(
    data_path=data_path, dataset_name=config['dataset']['dataset_name'])
# train_dataset = dataset.get_dataset(train_test='train')
val_dataset = dataset.get_dataset(train_test='val')
tst_dataset = dataset.get_dataset(train_test = 'test')
batch_size = 4
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, )
tst_loader = DataLoader(tst_dataset, batch_size = batch_size, shuffle=False, )

Load CODEmel dataset!
train size: 35995
val size: 2006
tst size: 1999
total size: 40000
Apply Val-stage Transform!
val dataset length:  2006
Apply Val-stage Transform!
test dataset length:  1999


# builder

In [6]:
model = ECGCLIP(config['network'])

In [7]:
model = model.to(device).eval()
ckpt = torch.load(model_checkpoints_folder + config['wandb_name'] + f'_bestZeroShotAll_ckpt.pth', map_location='cpu')
model.load_state_dict(ckpt)

C:\Users\katri\AppData\Local\Temp\ipykernel_18132\2575387603.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_checkpoints_folder + config['wandb_n

<All keys matched successfully>

# zeroshot

In [8]:
# # text_normal = 'ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: duracao normal. conclusao: 1- ecg dentro dos limites da normalidade. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_1davb = 'ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao aumentadata >200 ms. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: duracao normal. conclusao: 1- bloqueio atrioventricular de primeiro grau. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_rbbb = 'ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: eixo e amplitudes normais. duracao aumentada. morfologia brd. st e onda t: alteracoes secundarias ao brd. qtc: duracao normal. conclusao: 1- bloqueio de ramo direito. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_lbbb = 'ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: eixo e amplitudes normais. duracao aumentada. morfologia bre. st e onda t: alteracoes secundarias ao bre. qtc: duracao normal. conclusao: 1- bloqueio de ramo esquerdo. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_sb = 'ritmo sinusal regular com frequencia fc de bpm. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: ms duracao normal. conclusao: 1- bradicardia sinusal (fc bpm). o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_af = 'ritmo irregular. sem desvio de eixo. ausencia de onda p. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: duracao normal. conclusao: 1- fibrilacao atrial. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_st = 'ritmo sinusal regular com frequencia fc de bpm. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: ms duracao normal. conclusao: 1- taquicardia sinusal (fc bpm). o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'

# texts = [text_1davb, text_rbbb, text_lbbb, text_sb, text_af, text_st]
# num_classes = len(texts)

In [9]:
# eixos = ['Sem desvio de eixo.']
# ondaps = ['Onda P: amplitude e duração normais.', 
#           'Onda P: ausente com R-R irregular.', 'AUSENCIA DE ONDA P.', ] # af
# pris = ['PRi: duracao normal.', 
#         'PRi: duracao: >200ms.', 'PRI: DURACAO AUMENTADA (>200 MS).', ] # 1davb
# qrss = ['QRS: DURACAO, EIXO, MORFOLOGIA E AMPLITUDE NORMAIS.', 
#         'QRS: DURACAO AUMENTADA, MORFOLOGIA DE BRD. AMPLITUDE NORMAL.', 'QRS: DURACAO AUMENTADA, MORFOLOGIA DE bloqueio do ramo direito. AMPLITUDE NORMAL.', 'QRS: Eixo e Amplitudes normais. Duracao aumentada e rsR em V1 e onda S empastada nas derivacoes a esquerda.', 'QRS: EIXO E AMPLITUDES NORMAIS. DURACAO AUMENTADA E RSR&APOS; EM V1 E ONDA S EMPASTADA NAS DERIVACOES A ESQUERDA.', 'QRS: Eixo e Amplitudes normais. Duracao (130 ms). Morfologia: BRD: rsR em V1 e onda S empastada nas derivacoes a esquerda.', # rbbb
#         'QRS: MORFOLOGIA DE BRE. DURACAO AUMENTADA (150 MS).', 'QRS: com duração aumentada > 120 ms e morfologia de bloqueio do ramo esquerdo.', ] # lbbb
# # sts = ['ST: SEM SUPRA OU INFRADESNIVELAMENTO.', 
# #        ]
# # ondats = ['ONDA T: MORFOLOGIA HABITUAL.', 
# #           ]
# stsandondats = ['ST e Onda T: normal.', 'ST: SEM SUPRA OU INFRADESNIVELAMENTO. ONDA T: MORFOLOGIA HABITUAL.'
#                 'ST e Onda T: alteracoes secundarias ao BRD.', 'ST e Onda T: alteracoes secundarias ao bloqueio do ramo direito.', # rbbb
#                 'ST e Onda T: alterações secundarias ao BRE.', 'ST e Onda T: alteracoes secundarias ao bloqueio do ramo esquerdo.'] # lbbb
# qtcs = ['QTi: duracao normal.', 'QTc: duração normal.', 
#         'QTI: 370MS DURACAO NORMAL.', 'QTi: 434ms duracao normal.'] # sb
# conclusoes = ['Conclusao: 1- Bloqueio atrioventricular de primeiro grau.', 'CONCLUSAO: 1- BLOQUEIO ATRIOVENTRICULAR 1 GRAU.', 'Conclusao: 1- Bloqueio AV de primeiro grau.', # 1davb
#               'Conclusao: 1- Bloqueio de ramo direito.', # rbbb
#               'Conclusao: 1- Bloqueio de ramo esquerdo.', # lbbb
#               'CONCLUSAO: 1- BRADICARDIA SINUSAL (FC=49 BPM).', 'CONCLUSAO: 1- BRADICARDIA SINUSAL.', # sb
#               'CONCLUSAO: 1- FIBRILACAO ATRIAL.', # af
#               'Conclusao: 1- Taquicardia sinusal.', 'Conclusao: 1- Taquicardia Sinusal (FC= 103bpm)', ] # st

In [10]:
qtcs = ['QTi: duracao normal.', 'QTc: duração normal.']
stsandondats = ['ST e Onda T: normal.', 'ST: SEM SUPRA OU INFRADESNIVELAMENTO. ONDA T: MORFOLOGIA HABITUAL.']

# 1davb
pris = ['PRi: duracao: >200ms.', 'PRI: DURACAO AUMENTADA (>200 MS).']
conclusoes_1davb = ['Conclusao: 1- Bloqueio atrioventricular de primeiro grau.', 'CONCLUSAO: 1- BLOQUEIO ATRIOVENTRICULAR 1 GRAU.', 'Conclusao: 1- Bloqueio AV de primeiro grau.']

# rbbb
qrss_rbbb = ['QRS: DURACAO AUMENTADA, MORFOLOGIA DE BRD. AMPLITUDE NORMAL.', 'QRS: DURACAO AUMENTADA, MORFOLOGIA DE bloqueio do ramo direito. AMPLITUDE NORMAL.', 'QRS: Eixo e Amplitudes normais. Duracao aumentada e rsR em V1 e onda S empastada nas derivacoes a esquerda.', 'QRS: EIXO E AMPLITUDES NORMAIS. DURACAO AUMENTADA E RSR&APOS; EM V1 E ONDA S EMPASTADA NAS DERIVACOES A ESQUERDA.', 'QRS: Eixo e Amplitudes normais. Duracao (130 ms). Morfologia: BRD: rsR em V1 e onda S empastada nas derivacoes a esquerda.'] 
stsandondats_lbbb = ['ST e Onda T: alteracoes secundarias ao BRD.', 'ST e Onda T: alteracoes secundarias ao bloqueio do ramo direito.']

# lbbb
qrss_lbbb = ['QRS: MORFOLOGIA DE BRE. DURACAO AUMENTADA (150 MS).', 'QRS: com duração aumentada > 120 ms e morfologia de bloqueio do ramo esquerdo.']
stsandondats_rbbb = ['ST e Onda T: alteracoes secundarias ao BRD.', 'ST e Onda T: alteracoes secundarias ao bloqueio do ramo direito.']

# sb
conclusoes_sb = ['CONCLUSAO: 1- BRADICARDIA SINUSAL (FC=49 BPM).', 'CONCLUSAO: 1- BRADICARDIA SINUSAL.']
qtcs_sb = ['QTI: 370MS DURACAO NORMAL.', 'QTi: 434ms duracao normal.']

# af
ondaps = ['Onda P: ausente com R-R irregular.', 'AUSENCIA DE ONDA P.']

# st
conclusoes_st = ['Conclusao: 1- Taquicardia sinusal.', 'Conclusao: 1- Taquicardia Sinusal (FC= 103bpm)']

In [11]:
text_1davb = ['ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao aumentadata >200 ms. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: duracao normal. conclusao: 1- bloqueio atrioventricular de primeiro grau. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.']
for pri in pris:
    for standondat in stsandondats:
        for qtc in qtcs:
            for conclusao in conclusoes_1davb:
                text_1davb.append(f'Ritmo sinusal regular. Sem desvio de eixo. {pri} QRS: DURACAO, EIXO, MORFOLOGIA E AMPLITUDE NORMAIS. {standondat} {qtc} {conclusao}')

text_rbbb = ['ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: eixo e amplitudes normais. duracao aumentada. morfologia brd. st e onda t: alteracoes secundarias ao brd. qtc: duracao normal. conclusao: 1- bloqueio de ramo direito. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.']
for qrs in qrss_rbbb:
    for standondat in stsandondats_rbbb:
        for qtc in qtcs:
            text_rbbb.append(f'Ritmo sinusal regular. Sem desvio de eixo. Onda P: amplitude e duração normais. PRi: duracao normal. {qrs} {standondat} {qtc} Conclusao: 1- Bloqueio de ramo direito.')

text_lbbb = ['ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: eixo e amplitudes normais. duracao aumentada. morfologia bre. st e onda t: alteracoes secundarias ao bre. qtc: duracao normal. conclusao: 1- bloqueio de ramo esquerdo. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.']
for qrs in qrss_lbbb:
    for standondat in stsandondats_lbbb:
        for qtc in qtcs:
            text_lbbb.append(f'Ritmo sinusal regular. Sem desvio de eixo. Onda P: amplitude e duração normais. PRi: duracao normal. {qrs} {standondat} {qtc} Conclusao: 1- Bloqueio de ramo esquerdo.')

text_sb = ['ritmo sinusal regular com frequencia fc de bpm. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: ms duracao normal. conclusao: 1- bradicardia sinusal (fc bpm). o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.']
for standondat in stsandondats:
    for qtc in qtcs_sb:
        for conclusao in conclusoes_sb:
            text_sb.append(f'ritmo sinusal regular com frequencia fc de bpm. Sem desvio de eixo. Onda P: amplitude e duração normais. PRi: duracao normal. QRS: DURACAO, EIXO, MORFOLOGIA E AMPLITUDE NORMAIS. {standondat} {qtc} {conclusao}')

text_af = ['ritmo irregular. sem desvio de eixo. ausencia de onda p. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: duracao normal. conclusao: 1- fibrilacao atrial. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.']
for ondap in ondaps:
    for standondat in stsandondats:
        for qtc in qtcs:
            text_af.append(f'Ritmo irregular. Sem desvio de eixo. {ondap} PRi: duracao normal. QRS: DURACAO, EIXO, MORFOLOGIA E AMPLITUDE NORMAIS. {standondat} {qtc} CONCLUSAO: 1- FIBRILACAO ATRIAL.')

text_st = ['ritmo sinusal regular com frequencia fc de bpm. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: ms duracao normal. conclusao: 1- taquicardia sinusal (fc bpm). o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.']
for standondat in stsandondats:
    for qtc in qtcs_sb:
        for conclusao in conclusoes_st:
            text_st.append(f'ritmo sinusal regular com frequencia fc de bpm. Sem desvio de eixo. Onda P: amplitude e duração normais. PRi: duracao normal. QRS: DURACAO, EIXO, MORFOLOGIA E AMPLITUDE NORMAIS. {standondat} {qtc} {conclusao}')

texts = text_1davb + text_rbbb + text_lbbb + text_sb + text_af + text_st
num_classes = 6

In [12]:
pre = 0
pos = len(text_1davb)
slice_1davb = slice(pre,pos)

pre = pos
pos += len(text_rbbb)
slice_rbbb = slice(pre,pos)

pre = pos
pos += len(text_lbbb)
slice_lbbb = slice(pre,pos)

pre = pos
pos += len(text_sb)
slice_sb = slice(pre,pos)

pre = pos
pos += len(text_af)
slice_af = slice(pre,pos)

pre = pos
pos += len(text_st)
slice_st = slice(pre,pos)

slices = [slice_1davb, slice_rbbb, slice_lbbb, slice_sb, slice_af, slice_st]

## ecg

In [13]:
with torch.no_grad():
    # text synthesis
    zeroshot_weights = []
    for text in tqdm(texts):
        text = model._tokenize([text.lower()])

        class_embeddings = model.get_text_emb(text.input_ids.to(device=device), text.attention_mask.to(device=device)) # embed with text encoder
        class_embeddings = model.proj_t(class_embeddings) # embed with text encoder

        # normalize class_embeddings
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        # average over templates 
        class_embedding = class_embeddings.mean(dim=0) 
        # norm over new averaged templates
        class_embedding /= class_embedding.norm()

        zeroshot_weights.append(class_embedding)
    zeroshot_weights = torch.stack(zeroshot_weights, dim=1)

    # val thresholds
    thresholds = np.arange(0, 1.01, 0.01)
    predictions = {thresh: [[] for _ in range(num_classes)] for thresh in thresholds}
    true_labels_dict = [[] for _ in range(num_classes)]
    for data in tqdm(val_loader):
        # read
        # report = data['raw_text']
        ecg = data['ecg'].to(torch.float32).to(device).contiguous()
        label = data['label'].float().to(device)
        exam_id = data['exam_id'].to(device)

        # predict
        ecg_emb = model.ext_ecg_emb(ecg)
        ecg_emb /= ecg_emb.norm(dim=-1, keepdim=True)

        # obtain logits (cos similarity)
        logits = ecg_emb @ zeroshot_weights
        logits = torch.squeeze(logits, 0) # (N, num_classes)
        logits = torch.stack([torch.max(logits[:, s], axis = 1).values for s in slices]).T

        # norm_logits = (logits - logits.mean()) / (logits.std())
        # norm_logits = (logits - logits.mean(axis = 1).unsqueeze(1)) / (logits.std(axis = 1).unsqueeze(1))
        # probs = torch.sigmoid(norm_logits)
        probs = torch.sigmoid(logits)

        for class_idx in range(num_classes):
            for thresh in thresholds:
                predicted_binary = (probs[:, class_idx] >= thresh).float()
                predictions[thresh][class_idx].extend(predicted_binary.cpu().numpy())
            true_labels_dict[class_idx].extend(label[:, class_idx].cpu().numpy())
    best_f1s, best_thresholds = find_best_thresholds(predictions, true_labels_dict, thresholds)

    # test
    y_pred = []
    all_binary_results = []
    all_true_labels = []
    for data in tqdm(tst_loader):
        # read
        # report = data['raw_text']
        ecg = data['ecg'].to(torch.float32).to(device).contiguous()
        label = data['label'].float().to(device)
        exam_id = data['exam_id'].to(device)

        # predict
        ecg_emb = model.ext_ecg_emb(ecg)
        ecg_emb /= ecg_emb.norm(dim=-1, keepdim=True)

        # obtain logits (cos similarity)
        logits = ecg_emb @ zeroshot_weights
        logits = torch.squeeze(logits, 0) # (N, num_classes)
        logits = torch.stack([torch.max(logits[:, s], axis = 1).values for s in slices]).T

        # norm_logits = (logits - logits.mean()) / (logits.std())
        # norm_logits = (logits - logits.mean(axis = 1).unsqueeze(1)) / (logits.std(axis = 1).unsqueeze(1))
        # probs = torch.sigmoid(norm_logits)
        probs = torch.sigmoid(logits)

        binary_result = torch.zeros_like(probs)
        for i in range(len(best_thresholds)):
            binary_result[:, i] = (probs[:, i] >= best_thresholds[i]).float()

        y_pred.append(logits)
        all_binary_results.append(binary_result)
        all_true_labels.append(label)
        
    y_pred = torch.cat(y_pred, dim=0)
    all_binary_results = torch.cat(all_binary_results, dim=0)
    all_true_labels = torch.cat(all_true_labels, dim=0)

100%|██████████| 500/500 [00:59<00:00,  8.45it/s]


In [14]:
metrics_table(all_binary_results, all_true_labels)

{'Accuracy': [0.832416208104052,
  0.9719859929964982,
  0.9899949974987494,
  0.9874937468734367,
  0.9779889944972486,
  0.9469734867433717,
  0.8589294647323662],
 'F1 Score': [np.float64(0.09703504043126684),
  np.float64(0.5882352941176471),
  np.float64(0.7560975609756098),
  np.float64(0.5454545454545454),
  np.float64(0.26666666666666666),
  np.float64(0.3614457831325301),
  np.float64(0.9166173861620343)],
 'AUC ROC': [np.float64(0.6451123021949975),
  np.float64(0.8806065547368844),
  np.float64(0.9153511309474612),
  np.float64(0.7558288114825835),
  np.float64(0.6130669061517737),
  np.float64(0.8911954491424691),
  np.float64(0.8324824249552272)]}

## text

In [16]:
with torch.no_grad():
    # # text synthesis
    # zeroshot_weights = []
    # for text in tqdm(texts):
    #     text = model._tokenize([text.lower()])

    #     class_embeddings = model.get_text_emb(text.input_ids.to(device=device), text.attention_mask.to(device=device)) # embed with text encoder
    #     class_embeddings = model.proj_t(class_embeddings) # embed with text encoder

    #     # normalize class_embeddings
    #     class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
    #     # average over templates 
    #     class_embedding = class_embeddings.mean(dim=0) 
    #     # norm over new averaged templates
    #     class_embedding /= class_embedding.norm()

    #     zeroshot_weights.append(class_embedding)
    # zeroshot_weights = torch.stack(zeroshot_weights, dim=1)

    # val thresholds
    thresholds = np.arange(0, 1.01, 0.01)
    predictions = {thresh: [[] for _ in range(num_classes)] for thresh in thresholds}
    true_labels_dict = [[] for _ in range(num_classes)]
    for data in tqdm(val_loader):
        # read
        report = data['raw_text']
        # ecg = data['ecg'].to(torch.float32).to(device).contiguous()
        label = data['label'].float().to(device)
        exam_id = data['exam_id'].to(device)

        # predict
        text = model._tokenize(report)
        class_embeddings = model.get_text_emb(text.input_ids.to(device=device), text.attention_mask.to(device=device)) # embed with text encoder
        class_embeddings = model.proj_t(class_embeddings) # embed with text encoder

        # normalize class_embeddings
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        # average over templates 
        class_embedding = class_embeddings
        # norm over new averaged templates
        class_embedding /= class_embedding.norm()

        # obtain logits (cos similarity)
        logits = class_embedding @ zeroshot_weights
        logits = torch.squeeze(logits, 0) # (N, num_classes)
        logits = torch.stack([torch.max(logits[:, s], axis = 1).values for s in slices]).T

        # norm_logits = (logits - logits.mean()) / (logits.std())
        norm_logits = (logits - logits.mean(axis = 1).unsqueeze(1)) / (logits.std(axis = 1).unsqueeze(1))
        probs = torch.sigmoid(norm_logits)

        for class_idx in range(num_classes):
            for thresh in thresholds:
                predicted_binary = (probs[:, class_idx] >= thresh).float()
                predictions[thresh][class_idx].extend(predicted_binary.cpu().numpy())
            true_labels_dict[class_idx].extend(label[:, class_idx].cpu().numpy())
    best_f1s, best_thresholds = find_best_thresholds(predictions, true_labels_dict, thresholds)

    # test
    y_pred = []
    all_binary_results = []
    all_true_labels = []
    for data in tqdm(tst_loader):
        # read
        report = data['raw_text']
        # ecg = data['ecg'].to(torch.float32).to(device).contiguous()
        label = data['label'].float().to(device)
        exam_id = data['exam_id'].to(device)

        # predict
        text = model._tokenize(report)
        class_embeddings = model.get_text_emb(text.input_ids.to(device=device), text.attention_mask.to(device=device)) # embed with text encoder
        class_embeddings = model.proj_t(class_embeddings) # embed with text encoder

        # normalize class_embeddings
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        # average over templates 
        class_embedding = class_embeddings
        # norm over new averaged templates
        class_embedding /= class_embedding.norm()

        # obtain logits (cos similarity)
        logits = class_embedding @ zeroshot_weights
        logits = torch.squeeze(logits, 0) # (N, num_classes)
        logits = torch.stack([torch.max(logits[:, s], axis = 1).values for s in slices]).T

        # norm_logits = (logits - logits.mean()) / (logits.std())
        norm_logits = (logits - logits.mean(axis = 1).unsqueeze(1)) / (logits.std(axis = 1).unsqueeze(1))
        probs = torch.sigmoid(norm_logits)

        binary_result = torch.zeros_like(probs)
        for i in range(len(best_thresholds)):
            binary_result[:, i] = (probs[:, i] >= best_thresholds[i]).float()

        y_pred.append(logits)
        all_binary_results.append(binary_result)
        all_true_labels.append(label)
        
    y_pred = torch.cat(y_pred, dim=0)
    all_binary_results = torch.cat(all_binary_results, dim=0)
    all_true_labels = torch.cat(all_true_labels, dim=0)

100%|██████████| 500/500 [14:04<00:00,  1.69s/it]


In [17]:
metrics_table(all_binary_results, all_true_labels)

{'Accuracy': [0.8849424712356178,
  0.9644822411205602,
  0.950975487743872,
  0.9829914957478739,
  0.9479739869934968,
  0.966983491745873,
  0.8054027013506754],
 'F1 Score': [np.float64(0.15441176470588236),
  np.float64(0.5170068027210885),
  np.float64(0.3287671232876712),
  np.float64(0.48484848484848486),
  np.float64(0.07142857142857142),
  np.float64(0.43103448275862066),
  np.float64(0.8810033649434078)],
 'AUC ROC': [np.float64(0.7086459928534966),
  np.float64(0.8576619559528124),
  np.float64(0.8026627544976169),
  np.float64(0.7705321197269386),
  np.float64(0.5399940128723245),
  np.float64(0.8332130525839135),
  np.float64(0.8089332014648098)]}